In [16]:
len(w2v.vocab)

2861774

In [15]:
from gensim.models.keyedvectors import KeyedVectors
import faiss
import numpy as np
from time import time
import codecs


def build_vector_index(w2v_fpath):
    w2v = KeyedVectors.load_word2vec_format(w2v_fpath, binary=False, unicode_errors='ignore')
    w2v.init_sims(replace=True)
    index = faiss.IndexFlatIP(w2v.vector_size)
    index.add(w2v.syn0norm)

    return index


def compute_neighbours(index, nns_fpath, neighbors=200):
    tic = time()
    with codecs.open(nns_fpath, "w", "utf-8") as output:
        X = w2v.syn0norm
        D, I = index.search(X, neighbors + 1)

        j = 0
        for _D, _I in zip(D, I):
            for n, (d, i) in enumerate(zip(_D.ravel(), _I.ravel())):
                if n > 0:
                    output.write("{}\t{}\t{:f}\n".format(w2v.index2word[j], w2v.index2word[i], d))
            j += 1

        print("Word graph:", nns_fpath)
        print("Elapsed: {:d} sec.".format(time() - tic))
        

w2v_fpath = "/home/panchenko/sensegram/model/wikipedia-phrases-list-dim300.w2v"
nns_fpath = "model/wikipedia-phrases-list-dim300.neighbors.all2"

# index = build_vector_index(w2v_fpath)
compute_neighbours(index, nns_fpath, neighbors=200)

Word graph: model/wikipedia-phrases-list-dim300.neighbors.all2


ValueError: Unknown format code 'd' for object of type 'float'